## Classification of the CIFAR-10 Dataset

This is a reflective report on the universal deep learning workflow and how it is applied it in the image classification of the CIFAR-10 Dataset.

---

1. Defining the problem and Assembling a Dataset
    - The Problem
    - The Data
    - Naive Baseline
2. Choosing a Measure of Success
    - Primary Metrics
    - Additional Metrics
3. Deciding on an Evaluation Protocol
    - 
4. Preparing the Data
5. Developing and Comparing Two Small Models
6. Scaling up: Overfitting Experiment
7. Regularizing and Hyperparameter Tuning
8. Final Model and Test Evaluation